# Applied Data Science Capstone Week 3 Assignment

## 1. Scraping Toronto Neighborhood data from a webpage

In [1]:
# import required libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

# get webpage content
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
webpage = requests.get(url)

# parse webpage content
soupcontent = BeautifulSoup(webpage.content,'html.parser')

# find the first/only table
toronto_FSA = soupcontent.find('tbody')

# select each row (defined as <tr> in an HTML table)
toronto_FSA_r = toronto_FSA.select('tr')

# get text in each row
toronto_FSA_rows = [r_number.get_text() for r_number in toronto_FSA_r]

# convert to a dataframe
toronto_df = pd.DataFrame(toronto_FSA_rows)

# split into multiple columns by searching \n
toronto_df = toronto_df[0].str.split('\n',expand=True)

# rename columns
toronto_df = toronto_df.rename(columns=toronto_df.iloc[0])

# drop the first row
toronto_df = toronto_df.drop(toronto_df.index[0])

# drop the rows where Borough is Not assigned
toronto_df = toronto_df[toronto_df.Borough!='Not assigned']

# put neighborhoods that have the same postcodes/boroughs in a single row
toronto_df = toronto_df.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

# check the row where Neighborhood is Not assigned
toronto_df[toronto_df['Neighborhood']=='Not assigned']

# replace each Not assigned in Column Neighborhood with the entry in the same row of Column Borough
for toronto_df_row in range(toronto_df.shape[0]):
    if toronto_df.iloc[toronto_df_row,2] == 'Not assigned':
        toronto_df.iloc[toronto_df_row,2] = toronto_df.iloc[toronto_df_row,1]

# check if there still exists Not assigned
'Not assigned' in toronto_df['Neighborhood'].values

print('the number of rows of the dataframe is:')
toronto_df.shape

the number of rows of the dataframe is:


(103, 3)

## 2. Add the dataframe with both Latitude and Longitude information

In [2]:
# get the csv file that has the geographical coordinates of each postal code
# get it as a dataframe
toronto_pc_geo = pd.read_csv('http://cocl.us/Geospatial_data')

# update the column name(s) so that they are consistent with the one we just created
toronto_pc_geo.rename(columns={'Postal Code': 'Postcode'},inplace=True)

# merge the two dataframes 
toronto_df_with_geo = pd.merge(toronto_df,toronto_pc_geo,on='Postcode')
toronto_df_with_geo

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


## 3. Explore and cluster the neighborhoods in Toronto

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [4]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

C:\Anaconda3\envs\bobcy\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of the City of Toronto are 43.653963, -79.387207.


In [5]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_with_geo['Latitude'], 
                                           toronto_df_with_geo['Longitude'], 
                                           toronto_df_with_geo['Borough'], 
                                           toronto_df_with_geo['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### get Foursquare information

In [6]:
CLIENT_ID = 'EMYLQ1KXDHYJHMPXWYJPZ3JXQWHA4JUNTQGI51MOOC3OURJS' # your Foursquare ID
CLIENT_SECRET = 'DFRHGTMANANT3GHBMLK4HOJVVQF25IUZ0WJIMVPH2SDUS4EB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EMYLQ1KXDHYJHMPXWYJPZ3JXQWHA4JUNTQGI51MOOC3OURJS
CLIENT_SECRET:DFRHGTMANANT3GHBMLK4HOJVVQF25IUZ0WJIMVPH2SDUS4EB


#### get Toronto Neighborhood

In [7]:
toronto_neighborhood = toronto_df_with_geo[toronto_df_with_geo['Borough'].str.contains('Toronto')]
toronto_neighborhood_df = toronto_neighborhood.reset_index(drop=True)
toronto_neighborhood_df

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


#### create Toronto map

In [37]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_neighborhood_df['Latitude'], 
                                           toronto_neighborhood_df['Longitude'], 
                                           toronto_neighborhood_df['Borough'], 
                                           toronto_neighborhood_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [9]:
# check the first neighborhood
toronto_neighborhood_df.loc[0, 'Neighborhood']

'The Beaches'

In [10]:
# get the first neighborhood's latitude and longitude values
neighbourhood_latitude = toronto_neighborhood_df.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = toronto_neighborhood_df.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = toronto_neighborhood_df.loc[0, 'Neighborhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


#### get the top 50 venues that are in 'The Beaches' within a radius of 800 meters

In [11]:
LIMIT = 50
radius = 800

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=EMYLQ1KXDHYJHMPXWYJPZ3JXQWHA4JUNTQGI51MOOC3OURJS&client_secret=DFRHGTMANANT3GHBMLK4HOJVVQF25IUZ0WJIMVPH2SDUS4EB&v=20180605&ll=43.67635739999999,-79.2930312&radius=800&limit=50'

In [12]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e1dec59dd0f85001b0db551'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 50,
  'suggestedBounds': {'ne': {'lat': 43.6835574072, 'lng': -79.28309474318483},
   'sw': {'lat': 43.66915739279999, 'lng': -79.30296765681517}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'dis

In [13]:
# borrow the get_category_type function from the Foursquare lab
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
# clean the json and structure it into a dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,Beaches Bake Shop,Bakery,43.680363,-79.289692
2,The Beech Tree,Gastropub,43.680493,-79.288846
3,Tori's Bakeshop,Vegetarian / Vegan Restaurant,43.672114,-79.290331
4,The Fox Theatre,Indie Movie Theater,43.672801,-79.287272


In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

50 venues were returned by Foursquare.


#### create a function to repeat the same process to all the neighborhoods in '%Toronto'

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=800):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
# get the venues
toronto_venues = getNearbyVenues(names=toronto_neighborhood_df['Neighborhood'],
                                 latitudes=toronto_neighborhood_df['Latitude'],
                                 longitudes=toronto_neighborhood_df['Longitude']
                                 )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

In [18]:
print(toronto_venues.shape)
toronto_venues.head()

(1704, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
2,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
3,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
4,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater


In [19]:
# check how many venues were returned for each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",50,50,50,50,50,50
Berczy Park,50,50,50,50,50,50
"Brockton, Exhibition Place, Parkdale Village",50,50,50,50,50,50
Business Reply Mail Processing Centre 969 Eastern,50,50,50,50,50,50
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",29,29,29,29,29,29
"Cabbagetown, St. James Town",50,50,50,50,50,50
Central Bay Street,50,50,50,50,50,50
"Chinatown, Grange Park, Kensington Market",50,50,50,50,50,50
Christie,50,50,50,50,50,50


#### check how many unique categories can be curated from all the returned venues

In [20]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 241 uniques categories.


#### digitize all neighborhoods by one hot encoding

In [21]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Noodle House,Opera House,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [22]:
toronto_onehot.shape

(1704, 242)

#### group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category

In [23]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Noodle House,Opera House,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.0,0.0,0.0,0.0,0.00,0.00,0.04,0.0,0.00,0.0,0.00,0.00,0.020000,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.000000,0.02,0.000000,0.02,0.04,0.00,0.0,0.02,0.02,0.0,0.0,0.0,0.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.040000,0.0,0.00,0.00,0.04,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.02,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.02,0.0,0.0,0.02,0.0,0.00,0.0,0.02,0.00,0.02,0.02,0.0,0.000000,0.00,0.0,

In [24]:
toronto_grouped.shape

(39, 242)

#### print each neighborhood along with the top 5 most common venues

In [25]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0              Café  0.08
1        Steakhouse  0.06
2      Concert Hall  0.04
3    Breakfast Spot  0.04
4  Asian Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1            Beer Bar  0.06
2                Café  0.06
3            Creperie  0.04
4  Seafood Restaurant  0.04


----Brockton, Exhibition Place, Parkdale Village----
                 venue  freq
0                 Café  0.08
1          Coffee Shop  0.06
2           Restaurant  0.06
3               Bakery  0.06
4  Arts & Crafts Store  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0  Fast Food Restaurant  0.08
1       Harbor / Marina  0.04
2    Italian Restaurant  0.04
3           Pizza Place  0.04
4           Coffee Shop  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Harbo

### put the above data into a dataframe

##### write a function to sort the venues in descending order

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### create the new dataframe and display the top 10 venues for each neighborhood

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Steakhouse,Hotel,Sushi Restaurant,Pizza Place,Breakfast Spot,American Restaurant,Coffee Shop,Concert Hall,Asian Restaurant
1,Berczy Park,Coffee Shop,Café,Beer Bar,Cheese Shop,Bakery,Seafood Restaurant,Farmers Market,Japanese Restaurant,Cocktail Bar,Creperie
2,"Brockton, Exhibition Place, Parkdale Village",Café,Restaurant,Bakery,Coffee Shop,Hotel,Arts & Crafts Store,Thrift / Vintage Store,Bar,Furniture / Home Store,Gift Shop
3,Business Reply Mail Processing Centre 969 Eastern,Fast Food Restaurant,Pizza Place,Bar,Italian Restaurant,Brewery,Grocery Store,Harbor / Marina,Burrito Place,Coffee Shop,Park
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Harbor / Marina,Airport Service,Airport Terminal,Boat or Ferry,Sculpture Garden,Airport Lounge,Coffee Shop,Airport Food Court,Airport Gate,Bar


### Cluster the neighborhoods

#### run k-means to cluster the neighborhood into 5 clusters

In [28]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 4, 4, 0, 4, 0, 4, 4, 0, 4, 4, 4, 0, 4, 0, 4, 0, 4, 0, 0,
       3, 0, 1, 4, 0, 4, 1, 2, 4, 4, 0, 0, 0, 0, 0, 0, 0])

### create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_neighborhood_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Bar,Breakfast Spot,Health Food Store,Coffee Shop,Bakery,Grocery Store,Sandwich Place,Japanese Restaurant,Caribbean Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Ice Cream Shop,Coffee Shop,Bakery,Café,Pub,Italian Restaurant,Yoga Studio,Donut Shop,Juice Bar
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Indian Restaurant,Coffee Shop,Grocery Store,Gym,Fast Food Restaurant,Brewery,Café,Sandwich Place,Burrito Place,Fish & Chips Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,Bakery,Bar,American Restaurant,Brewery,Latin American Restaurant,Ice Cream Shop,Seafood Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Gym / Fitness Center,Coffee Shop,Bus Line,Bookstore,Restaurant,Park,Café,Dog Run,Doner Restaurant,Dumpling Restaurant


## Visualize the clustering results

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], 
                                  toronto_merged['Longitude'], 
                                  toronto_merged['Neighborhood'], 
                                  toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### check clusters and summarize them

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Pub,Bar,Breakfast Spot,Health Food Store,Coffee Shop,Bakery,Grocery Store,Sandwich Place,Japanese Restaurant,Caribbean Restaurant
1,East Toronto,0,Greek Restaurant,Ice Cream Shop,Coffee Shop,Bakery,Café,Pub,Italian Restaurant,Yoga Studio,Donut Shop,Juice Bar
2,East Toronto,0,Indian Restaurant,Coffee Shop,Grocery Store,Gym,Fast Food Restaurant,Brewery,Café,Sandwich Place,Burrito Place,Fish & Chips Shop
3,East Toronto,0,Café,Coffee Shop,Italian Restaurant,Bakery,Bar,American Restaurant,Brewery,Latin American Restaurant,Ice Cream Shop,Seafood Restaurant
11,Downtown Toronto,0,Restaurant,Café,Coffee Shop,Japanese Restaurant,Park,Bakery,Beer Store,Thai Restaurant,Italian Restaurant,Diner
15,Downtown Toronto,0,Café,Restaurant,Coffee Shop,Bakery,BBQ Joint,Beer Bar,Italian Restaurant,Japanese Restaurant,Farmers Market,Hotel
16,Downtown Toronto,0,Coffee Shop,Café,Beer Bar,Cheese Shop,Bakery,Seafood Restaurant,Farmers Market,Japanese Restaurant,Cocktail Bar,Creperie
18,Downtown Toronto,0,Café,Steakhouse,Hotel,Sushi Restaurant,Pizza Place,Breakfast Spot,American Restaurant,Coffee Shop,Concert Hall,Asian Restaurant
19,Downtown Toronto,0,Hotel,Park,Aquarium,Coffee Shop,Baseball Stadium,Fried Chicken Joint,Brewery,Scenic Lookout,Plaza,Café
20,Downtown Toronto,0,Café,Coffee Shop,Hotel,Steakhouse,Japanese Restaurant,Restaurant,Deli / Bodega,Beer Bar,Gastropub,Ice Cream Shop


### Cluster 0:  locations where cafes are generally the most common venues

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, 
                     toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,1,Park,Grocery Store,Gym,Sandwich Place,Japanese Restaurant,Café,Thai Restaurant,Candy Store,Playground,Gym / Fitness Center
10,Downtown Toronto,1,Park,Trail,Candy Store,Grocery Store,Bank,Playground,Doner Restaurant,Diner,Discount Store,Dive Bar


### Cluster 1:  locations where parks are generally the most common venues

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, 
                     toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,2,Playground,Pool,Garden,Home Service,IT Services,Yoga Studio,Dog Run,Diner,Discount Store,Dive Bar


### Cluster 2:  locations where playgrounds are generally the most common venues

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, 
                     toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,3,Gym / Fitness Center,Coffee Shop,Bus Line,Bookstore,Restaurant,Park,Café,Dog Run,Doner Restaurant,Dumpling Restaurant


### Cluster 3:  locations where gyms are generally the most common venues

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, 
                     toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Central Toronto,4,Italian Restaurant,Pizza Place,Café,Coffee Shop,Sushi Restaurant,Pharmacy,Sporting Goods Shop,Food & Drink Shop,Gym,Fast Food Restaurant
6,Central Toronto,4,Coffee Shop,Sporting Goods Shop,Diner,Italian Restaurant,Skating Rink,Café,Spa,Electronics Store,Salon / Barbershop,Restaurant
7,Central Toronto,4,Italian Restaurant,Sushi Restaurant,Gym,Dessert Shop,Café,Coffee Shop,Indian Restaurant,Pizza Place,Cemetery,Salad Place
9,Central Toronto,4,Coffee Shop,Sushi Restaurant,Italian Restaurant,Spa,Sandwich Place,Gym,Restaurant,Café,Pub,Yoga Studio
12,Downtown Toronto,4,Coffee Shop,Thai Restaurant,Burger Joint,Gastropub,Gay Bar,Men's Store,Restaurant,Gym,Ethiopian Restaurant,Bubble Tea Shop
13,Downtown Toronto,4,Coffee Shop,Pub,Café,Bakery,Park,Italian Restaurant,Breakfast Spot,Theater,Mexican Restaurant,Cosmetics Shop
14,Downtown Toronto,4,Coffee Shop,Cosmetics Shop,Clothing Store,Italian Restaurant,Restaurant,Theater,Ramen Restaurant,Middle Eastern Restaurant,Gastropub,Beer Bar
17,Downtown Toronto,4,Coffee Shop,Chinese Restaurant,Bubble Tea Shop,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Café,Tea Room,Pizza Place,Breakfast Spot
23,Central Toronto,4,Park,Italian Restaurant,Coffee Shop,Asian Restaurant,Gastropub,Café,Japanese Restaurant,Bank,Bagel Shop,Bakery
27,Downtown Toronto,4,Harbor / Marina,Airport Service,Airport Terminal,Boat or Ferry,Sculpture Garden,Airport Lounge,Coffee Shop,Airport Food Court,Airport Gate,Bar


### Cluster 4:  locations where coffee shops and restaurants are generally the most common venues